In [2]:
import pandas as pd
import csv
import re

In [3]:
# names of files to read from
r_maxo_classes_with_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/maxo_classes_with_definitions.tsv'
r_chebi_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/chebi_definitions.tsv'

tsv_read_maxo = pd.read_csv(r_maxo_classes_with_definitionsTSV, sep='\t')
tsv_read_chebi = pd.read_csv(r_chebi_definitionsTSV, sep='\t')


maxo_id=list()
chebi_id=list()

print(tsv_read_maxo.head(), tsv_read_chebi.head())
    # write to files



                                            ?cls          ?xref  \
0  <http://purl.obolibrary.org/obo/MAXO_0000468>   NCIT:C159340   
1  <http://purl.obolibrary.org/obo/MAXO_0000066>    NCIT:C94624   
2  <http://purl.obolibrary.org/obo/MAXO_0000004>    NCIT:C15329   
3  <http://purl.obolibrary.org/obo/MAXO_0000746>  PMID:25006279   
4  <http://purl.obolibrary.org/obo/MAXO_0001006>    NCIT:C28133   

                                                ?def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...  
2  A diagnostic or treatment procedure performed ...  
3  A test to detect chromosomal breakage or aberr...  
4  The determination of the number of red blood c...                                               ?cls  \
0  <http://purl.obolibrary.org/obo/CHEBI_141566>   
1   <http://purl.obolibrary.org/obo/CHEBI_15646>   
2   <http://purl.obolibrary.org/obo/CHEBI_16945>   
3    <http://purl.obolibrary.org/obo/CHEBI_1602>   
4   <ht

In [4]:
tsv_read_chebi.columns

Index(['?cls', '?def'], dtype='object')

In [6]:
from pandas import DataFrame

x=()
mylist=[]
newlist=list()
#extract just the maxo_id
maxo_id = pd.DataFrame(tsv_read_maxo)
cols = ["?cls","?xref","?def"]
maxo_id = maxo_id[maxo_id.columns[0]]
for line in maxo_id:
    line=line.strip('/')
    x=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    x=[item.replace('_', ':') for item in x]
    mylist.append(x)
    maxo_df= DataFrame(mylist,columns=['Maxo_ID'])

maxo_id_def= maxo_df.join(tsv_read_maxo, lsuffix="_left", rsuffix="_right")
    
with open("maxo_xref_definitions.tsv",'wb') as out:
    maxo_id_def.to_csv('maxo_xref_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    
    

In [7]:
y=()
newlist=[]
#extract just the ncit_id
chebi_id = pd.DataFrame(tsv_read_chebi)
cols = ["?cls","?def"]
chebi_id = chebi_id[chebi_id.columns[0]]

for line in chebi_id:
    line=line.strip('/')
    y=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    y=[item.replace('_', ':') for item in y]
    newlist.append(y)
    chebi_df= DataFrame(newlist,columns=['CHEBI_ID'])
chebi_id_def= chebi_df.join(tsv_read_chebi, lsuffix="_left", rsuffix="_right")
print(chebi_id_def.head(2))
    
with open("chebi_definitions.tsv",'wb') as out:
    chebi_id_def.to_csv('chebi_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    

       CHEBI_ID                                           ?cls  \
0  CHEBI:141566  <http://purl.obolibrary.org/obo/CHEBI_141566>   
1   CHEBI:15646   <http://purl.obolibrary.org/obo/CHEBI_15646>   

                                                ?def  
0  An enone that is hex-3-ene in which the methyl...  
1       The 20-hydroxy derivative of leukotriene B4.  


ncit_id_def.info

In [8]:

maxo_id_def.columns = ["Maxo_ID","?cls","ID","?def"]
print(maxo_id_def.head())

        Maxo_ID                                           ?cls             ID  \
0  MAXO:0000468  <http://purl.obolibrary.org/obo/MAXO_0000468>   NCIT:C159340   
1  MAXO:0000066  <http://purl.obolibrary.org/obo/MAXO_0000066>    NCIT:C94624   
2  MAXO:0000004  <http://purl.obolibrary.org/obo/MAXO_0000004>    NCIT:C15329   
3  MAXO:0000746  <http://purl.obolibrary.org/obo/MAXO_0000746>  PMID:25006279   
4  MAXO:0001006  <http://purl.obolibrary.org/obo/MAXO_0001006>    NCIT:C28133   

                                                ?def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...  
2  A diagnostic or treatment procedure performed ...  
3  A test to detect chromosomal breakage or aberr...  
4  The determination of the number of red blood c...  


In [9]:
maxo_id_list= []
maxo_def_list= []
maxo_def_xref_list= []
chebi_id_list=[]
chebi_def_list= []
for index, row in maxo_id_def.iterrows():
    if row[2].startswith("CHEBI:"):
        for index, rows in chebi_id_def.iterrows():
            #determine if the the MAXO def xref matches the Chebi ID
            if row[2] == rows[0]:
                maxo_id_list.append(row[0])
                maxo_def_list.append(row[3])
                maxo_def_xref_list.append(row[2])
                chebi_id_list.append(rows[0])
                chebi_def_list.append(rows[2])
            else:
                continue



In [10]:

maxo_chebi_def_df=pd.DataFrame(list(zip(maxo_id_list, maxo_def_list, maxo_def_xref_list, chebi_id_list, chebi_def_list)), columns=["maxo_id","maxo_def", "maxo_def_xref","chebi_id", "chebi_def"])
print(maxo_chebi_def_df.head())

with open("maxo_chebi_def.tsv",'wb') as out:
    maxo_chebi_def_df.to_csv('maxo_chebi_def.tsv', encoding='utf-8', sep='\t', index=False)
    


        maxo_id                                           maxo_def  \
0  MAXO:0000269  H2-receptor antagonists are the drugs that sel...   
1  MAXO:0001026  An agent that selectively binds to and activat...   
2  MAXO:0000445  A drug that prevents fibrinolysis or lysis of ...   
3  MAXO:0000220  Treatment with a class of steroid hormones tha...   
4  MAXO:0000446  A synthetic analogue of vasopressin in which 3...   

  maxo_def_xref     chebi_id  \
0   CHEBI:37961  CHEBI:37961   
1   CHEBI:35522  CHEBI:35522   
2   CHEBI:48675  CHEBI:48675   
3   CHEBI:24261  CHEBI:24261   
4    CHEBI:4450   CHEBI:4450   

                                           chebi_def  
0  H2-receptor antagonists are the drugs that sel...  
1  An agent that selectively binds to and activat...  
2  A drug that prevent fibrinolysis or lysis of a...  
3  Glucocorticoids are a class of steroid hormone...  
4  A synthetic analogue of vasopressin in which 3...  


[]
